In [ ]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

In [ ]:
pip install -U weaviate-client  # For beta versions: `pip install --pre -U "weaviate-client==4.*"`

In [ ]:
from google.colab import userdata
WEAVIATE_CLUSTER=userdata.get('WEAVIATE_CLUSTER')

In [ ]:
from google.colab import userdata
WEAVIATE_API_KEY=userdata.get('WEAVIATE_API_KEY')

In [ ]:
import weaviate

# Directly assign values to these variables
WEAVIATE_URL = WEAVIATE_CLUSTER  # Replace with the actual URL for Weaviate Cloud
WEAVIATE_API_KEY = WEAVIATE_API_KEY  # Replace with your actual API key

# Connect to Weaviate Cloud using the correct method for v4
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,  # URL for your Weaviate Cloud instance
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY)  # API key for authentication
)

# Now you can interact with Weaviate using `client`


In [ ]:
# from langchain.vectorstores import Weaviate
# import weaviate

# WEAVIATE_URL = WEAVIATE_CLUSTER
# WEAVIATE_API_KEY = WEAVIATE_API_KEY

# client = weaviate.Client(
#     url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
# )

In [ ]:

# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install langchain langchain_community

In [ ]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/finalPdf123.pdf", extract_images=True)
pages = loader.load()

In [ ]:
# Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [ ]:
docs[0]

In [ ]:
!pip install langchain_weaviate

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
# from langchain.vectorstores import Weaviate

# vector_db = Weaviate.from_documents(
#     docs, embeddings, client=client, by_text=False
# )

WEAVIATE_URL =WEAVIATE_CLUSTER  # Replace with the actual URL for Weaviate Cloud
WEAVIATE_API_KEY = WEAVIATE_API_KEY  # Replace with your actual API key
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,  # URL for your Weaviate Cloud instance
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY)  # AP
)
vector_db = WeaviateVectorStore.from_documents(docs,embeddings, client=weaviate_client)

In [ ]:
print(vector_db.similarity_search("ipc 212 case?", k=3)[0].page_content)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
# mistralai/Mistral-7B-Instruct-v0.1

In [ ]:
pip install -U langchain-huggingface

In [ ]:
from google.colab import userdata
sec_key=userdata.get("HUGGINGFACE_API")
print(sec_key)

In [ ]:
import os
os.environ["HUGGINGFACE_API"] = sec_key

In [ ]:
pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    token=sec_key,
    model_kwargs={"temperature": 0.7, "max_length": 128}
)

In [ ]:
# from transformers import pipeline

# # Set up pipeline with API key using use_auth_token

# generator = pipeline(
#     'text-generation',
#     model='mistralai/Mistral-7B-Instruct-v0.2',
#     use_auth_token=sec_key
# )

In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint
# llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2",model_kwargs={"temperature": 1, "max_length": 180,token:HUGGINGFACEHUB_API_TOKEN}
# )

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser=StrOutputParser()

In [ ]:
retriever=vector_db.as_retriever()

In [ ]:
output = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
while True:
    question = input("Ask Your Question: ")
    if question.lower() == "exit":
        break
    try:
        answer = output.invoke(question)
        print(answer)
    except Exception as e:
        print(f"An error occurred: {e}")